### Лекция 5. Шаблоны

<br />

##### Какая идея стоит за шаблонами

Ранее мы познакомились с возможностью перегрузки функций. Давайте вспомним её на примере swap:

In [ ]:
// # поменять местами два int
void my_swap(int& a, int& b)
{
    int tmp = a;
    a = b;
    b = tmp;    
}

// # поменять местами два short
void my_swap(short& a, short& b)
{
    short tmp = a;
    a = b;
    b = tmp;
}

// # поменять местами два float
void my_swap(float& a, float& b)
{
    float tmp = a;
    a = b;
    b = tmp;
}

...

Вечер начинает быть томным ...

Для решения проблем написания одинакового кода придуманы шаблоны:

In [ ]:
// # напишем шаблон - как должна выглядеть 
template<typename T>
void my_swap(T& a, T& b)
{
    Type tmp = a;
    a = b;
    b = tmp;
}

In [ ]:
int a = 3, b = 5;

// # вызов my_swap(int&, int&), тип T выводится компилятором автоматически
my_swap(a, b); 

// # вызов my_swap(int&, int&), тип T указывается программистом явно
my_swap<int>(a, b);


float x = 3.f, y = 5.f;
my_swap(x, y);
my_swap<float>(x, y);

Важное свойство шаблонов по сравнению с перегрузкой функций: шаблон компилируется только тогда, когда он вызывается. И компилируется только для тех типов, для которых он вызывается.

Показать пример на godbolt.org, позакомментировать функции, продемонстрировать разницу в выхлопе компилятора.

In [ ]:
#include <string>

template<typename T>
void __attribute__ ((noinline)) myswap(T& a, T& b)
{
    T tmp = a;
    a = b;
    b = tmp;
}

int main()
{
    int i1 = 3, i2 = 5;
    myswap(i1, i2);

    float f1 = 3.f, f2 = 5.f;
    myswap(f1, f2);

    double d1 = 3., d2 = 5.;
    myswap(d1, d2);

    std::string s1 = "abc", s2 = "def";
    myswap(s1, s2);

    return 0;
}

Особенности шаблонов по сравнению с перегруженными функциями:
* компилируется только то, что инстанциируется в коде
* компилируется столько раз, в скольки единицах трансляции инстанциируется:
    * можно в одном cpp-файле 10 раз позвать myswap(int&, int&) - эта функция скомпилируется единожды
    * можно в 10 cpp-файлах один раз позвать myswap(int&, int&) - эта функция скомпилируется 10 раз
* накладные расходы во время компиляции на кодогенерацию при истанциации
* позволяет компилятору агрессивнее использовать оптимизации (если горе-программист не запретил инлайнинг как в примере выше). Раскомментировать `__attribute__((noinline))` из примера и показать какой код сгенерирует компилятор. Объяснить, почему.
* позволяет нарушать ODR

Коротко:

* (+) меньше кода
* (+) быстрее
* (-) дольше компилируется
* (-) сложнее писать

<br />

##### Специализация

Перегрузка функций позволяла сделать `myswap` у `std::string` более эффективно, без лишнего копирования памяти:

In [ ]:
void myswap(int& a, int& b) { ... }
void myswap(short& a, short& b) { ... }

void myswap(std::string& a, std::string& b)
{
    a.swap(b);
}

Шаблоны тоже позволяют специализировать поведение функций, если наложить на шаблонный параметр огранчение, например:

(закинуть этот код на godbolt, показать во что компилируется программа)

In [ ]:
#include <string>

template<typename T>
void __attribute__ ((noinline)) myswap(T& a, T& b)
{
    T tmp = a;
    a = b;
    b = tmp;
}

template<>
void __attribute__ ((noinline)) myswap<std::string>(std::string& a, std::string& b)
{
    a.swap(b);
}

int main()
{
    int i1 = 3, i2 = 5;
    myswap(i1, i2);

    float f1 = 3.f, f2 = 5.f;
    myswap(f1, f2);

    double d1 = 3., d2 = 5.;
    myswap(d1, d2);

    std::string s1 = "abc", s2 = "def";
    myswap(s1, s2);

    return 0;
}

Во-первых, шаблон может иметь несколько параметров, а во-вторых, параметры не обязаны быть типами. Они могут быть, например, целыми числами:

In [ ]:
template<int N, typename T>
T add_value(T x)
{
    return x + N;
}


int a = add_value<5>(100);

// # 1. шаблон специфицирован программистом частично, тип Т компилятор определит сам
// # 2. параметром шаблона выступает целое число.

Теперь мы разбираемся в шаблонах достаточно чтобы посчитать факториал во время компиляции на шаблонах (разобрать пример, показать результат в godbolt).

Примечание: C++ значительно эволюционировал, и больше во время компиляции таким образом вычисления не проводят. Пример исключительно ученический. Compile-time вычисления будут, возможно, рассмотрены в курсе далее.

In [ ]:
template<unsigned N>
unsigned factorial()
{
    return N * factorial<N - 1>();
}

template<>
unsigned factorial<0>()
{
    return 1;
}

unsigned f()
{
    return factorial<5>();
}

<br />

##### Шаблонные классы

Аналогично функциям, классы тоже могут быть шаблонными:

Пример структуры:

In [ ]:
// # N-мерный вектор из курса линейной алгебры типа T
template<typename T, int N>
struct VectorN
{
    T data[N];
};

// # в качестве примера запишем операции сложения и умножения для таких векторов

template<typename T, int N>
VectorN<T, N> operator +(const VectorN<T, N>& l, const VectorN<T, N>& r)
{
    VectorN<T, N> rv;
    for (int i = 0; i < N; ++i)
        rv.data[i] = l.data[i] + r.data[i];
    return rv;    
}

template<typename T, int N>
T operator * (const VectorN<T, N>& l, const VectorN<T, N>& r)
{
    T rv = 0;
    for (int i = 0; i < N; ++i)
        rv += l.data[i] * r.data[i];
    return rv;
}

Пример класса:

In [ ]:
template<typename T, int N>
class RoundRobinQueue
{
private:
    std::array<T, N> arr;
    int begin_ix = 0;
    int end_ix = 0;

public:
    static int next_ix(const int ix)
    {
        return (ix + 1) % N;
    }
    
    void push(const T& item)
    {
        arr[end_ix] = item;
        end_ix = next_ix(end_ix);
    }
    
    T pop()
    {
        T item = arr[begin_ix];
        begin_ix = next_ix(begin_ix);
        return item;
    }
    
    bool empty() const
    {
        return begin_ix == end_ix;
    }
    
    bool full() const
    {
        return next_ix(end_ix) == begin_ix;
    }
};

In [ ]:
RoundRobinQueue<int, 100> queue;
queue.push(1);
queue.push(2);
queue.push(3);
queue.pop();  // # 1
queue.pop();  // # 2
queue.pop();  // # 3

А ещё класс может иметь шаблонный метод, почему бы и нет.

In [ ]:
#include <array>
#include <string>

template<typename T, int N>
class RoundRobinQueue
{
private:
    std::array<T, N> arr;
    int begin_ix = 0;
    int end_ix = 0;

public:
    static int next_ix(const int ix)
    {
        return (ix + 1) % N;
    }
    
    template<typename U>
    void push(const U& item)
    {
        arr[end_ix] = item;
        end_ix = next_ix(end_ix);
    }
    
    T pop()
    {
        T item = arr[begin_ix];
        begin_ix = next_ix(begin_ix);
        return item;
    }
    
    bool empty() const
    {
        return begin_ix == end_ix;
    }
    
    bool full() const
    {
        return next_ix(end_ix) == begin_ix;
    }
};

std::string f()
{
    RoundRobinQueue<std::string, 100> queue;
    queue.push(std::string("run"))
    queue.push("Forest");
    queue.push("run");
    return queue.pop();
}

Закинуть пример на godbolt, показать, что генерируется 3 метода `push`, не забыть убрать оптимизации

Шаблонный конструктор - пожалуйста! Это ведь тоже метод

Шаблонный деструктор... пожалуй, нет, хватит.

<br />

##### Частичная специализация шаблонов (для классов)

Например, можно прописать alias:

In [ ]:
template<int N>
using RoundRobinStringsQueue = RoundRobinQueue<std::string, N>;


RoundRobinStringsQueue<100> queue;

А можно и изменить поведение класса

In [ ]:
template<int N>
class RoundRobinQueue<std::string, N>
{
private:
    std::string arr[N];  // # !!! here
    int begin_ix = 0;
    int end_ix = 0;

public:
    ...
    
    template<typename U>
    void push(const U& item)
    {
        std::cout << "push!";  // # !!! here
        arr[end_ix] = item;
        end_ix = next_ix(end_ix);
    }
    
    ...
};

Такие сложности могут пригодиться для compile-time полиморфизма. С его классическим применением - type traits-ами - познакомимся позже.

<br />

Во второй части курса про шаблоны:
* SFINAE
* variadic templates
* type traits
* tag dispatching (возможно)